### **<span style="color:green">VISÃO GERAL DA BASE DE DADOS: Titanic</span>**

Esta base de dados contém informações sobre os passageiros do Titanic, o famoso navio que afundou em 1912. Ela é amplamente utilizada para análises de sobrevivência e para entender os fatores que influenciaram a sobrevivência dos passageiros. Aqui estão algumas das variáveis incluídas:

- **PassengerId**: Identificador único de cada passageiro.
- **Survived**: Indica se o passageiro sobreviveu (1) ou não (0).
- **Pclass**: Classe do bilhete do passageiro (1ª, 2ª, 3ª).
- **Name**: Nome do passageiro.
- **Sex**: Sexo do passageiro.
- **Age**: Idade do passageiro.
- **SibSp**: Número de irmãos/cônjuges a bordo do Titanic.
- **Parch**: Número de pais/filhos a bordo do Titanic.
- **Ticket**: Número do bilhete.
- **Fare**: Tarifa paga pelo bilhete.
- **Cabin**: Número da cabine.
- **Embarked**: Porto de embarque (C = Cherbourg; Q = Queenstown; S = Southampton).

A meta é entender como diferentes características dos passageiros e suas condições de viagem podem influenciar a probabilidade de sobrevivência no naufrágio do Titanic. Essa base de dados pode ser usada para diversas análises, como identificar padrões de sobrevivência, visualizar tendências demográficas dos passageiros e realizar previsões sobre fatores que afetaram as chances de sobrevivência.

Link para a competição do Kaggle: https://www.kaggle.com/competitions/titanic/overview

---

### **<span style="color:green">IMPORTS E CONFIGURAÇÕES</span>**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, roc_curve, accuracy_score, precision_score, recall_score, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

#!pip install ipywidgets

In [2]:
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import (
    LogisticRegression,
    RidgeClassifier,
    SGDClassifier,
)

from sklearn.ensemble import (
    AdaBoostClassifier, 
    BaggingClassifier, 
    ExtraTreesClassifier, 
    RandomForestClassifier, 
    StackingClassifier, 
    GradientBoostingClassifier, 
    HistGradientBoostingClassifier,
    VotingClassifier
)

from sklearn.preprocessing import StandardScaler

### **<span style="color:green">FUNÇÕES DE PROCESSAMENTO DE DADOS</span>**

In [3]:
CAMINHO_BASE_TREINAMENTO = 'https://raw.githubusercontent.com/claytonsilva007/IDP/refs/heads/main/ml-01/dados/titanic/train.csv'
CAMINHO_BASE_TESTE = 'https://raw.githubusercontent.com/claytonsilva007/IDP/refs/heads/main/ml-01/dados/titanic/test.csv'

In [4]:
def carregar_dados(path: str):
  dados = pd.read_csv(path)
  return dados

In [5]:
def obter_treino_e_teste(df: pd.DataFrame, perc_treino: float = 0.8):
    treino, teste = train_test_split(df, train_size=perc_treino ,random_state=42)
    return treino, teste  

In [6]:
def transformar_sex_em_inteiro(df: pd.DataFrame):
    df.Sex = df.Sex.map({'male': 0, 'female': 1})
    return df

In [7]:
def adicionar_indicador_menor_idade(df: pd.DataFrame):
    df['isChild'] = df.Age.map(lambda age: 1 if age < 0 else 0)
    return df

In [8]:
def get_pronouns(name) -> str:
    if ", " in name and "." in name:
        return name.split(", ")[1].split(".")[0]
    return "None" 

In [9]:
def encoding_pronouns(df: pd.DataFrame) -> pd.DataFrame:
    df.Pronouns = df.Pronouns.map({'Mrs': 5, 'Miss': 4, 'Master': 3, 'Other': 2, 'Mr': 1}).fillna(df.Pronouns)
    # df = pd.get_dummies(data=df, columns=['Pronouns'], dtype=int)
    return df

In [10]:
def join_pronouns(df: pd.DataFrame) -> pd.DataFrame:
    df['Pronouns'] = df['Pronouns'].map({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'}).fillna(df['Pronouns'])
    df.Pronouns = df['Pronouns'].map({
        'Dr': 'Other', 
        'Rev': 'Other', 
        'Col': 'Other', 
        'Major': 'Other', 
        'Don': 'Other', 
        'Lady': 'Other', 
        'Sir': 'Other', 
        'Capt': 'Other', 
        'the Countess': 'Other', 
        'Jonkheer': 'Other',
        'Dona': 'Other',
    }).fillna(df['Pronouns'])
    
    return df

In [11]:
def process_embarked(df: pd.DataFrame) -> pd.DataFrame:
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])    
    df.Embarked = LabelEncoder().fit_transform(df['Embarked'])
    return df

In [12]:
def process_age(df: pd.DataFrame):
    df['Age'] = df['Age'].fillna(df.groupby(by=['Sex', 'Pclass'])['Age'].transform('mean'))
    return df

In [13]:
def calcular_roc_auc(y_true, y_pred):
    # Calcular a curva ROC
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    
    # Calcular a área sob a curva (AUC)
    roc_auc = auc(fpr, tpr)
    
    return roc_auc

In [14]:
def exibir_metricas_desempenho(ypred, ytrue):
    acuracia = accuracy_score(y_pred=ypred, y_true=ytrue)
    precisao = precision_score(y_pred=ypred, y_true=ytrue)
    revocacao = recall_score(y_pred=ypred, y_true=ytrue)
    roc_auc = calcular_roc_auc(y_pred=ypred, y_true=ytrue)

    print(f'Acurácia: {round(acuracia, 5)}')
    print(f'Precisão: {round(precisao, 5)}')
    print(f'Revocação: {round(revocacao, 5)}')
    print(f'ROC-AUC: {round(roc_auc, 5)}')

In [15]:
def plot_roc_auc(ytrue, ypred):
    # Gerar curva de precisão-revocação
    precision, recall, _ = precision_recall_curve(ytrue, ypred)

    plt.figure(figsize=(14, 7))

    # Subplot para a curva de precisão-revocação
    plt.subplot(1, 2, 1)
    plt.plot(recall, precision, marker='.', label='Precision-Recall Curve')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend()

    # Subplot para a curva ROC
    fpr, tpr, _ = roc_curve(ytrue, ypred)

    plt.subplot(1, 2, 2)
    plt.plot(fpr, tpr, marker='.', label='ROC Curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()

    # Mostrar os gráficos
    plt.show()

### **<span style="color:green">OBTENHA E PROCESSE OS DADOS</span>**

In [16]:
def obter_dataframe_processado(path: str):
    df = carregar_dados(path=path)
    df = process_age(df=df)
    df = adicionar_indicador_menor_idade(df)
    df = transformar_sex_em_inteiro(df)
    df['Pronouns'] = df.Name.map(get_pronouns)
    df = join_pronouns(df)
    df = encoding_pronouns(df=df)
    df = process_embarked(df=df)
    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
    return df 

In [17]:
df = obter_dataframe_processado(path=CAMINHO_BASE_TREINAMENTO)
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,isChild,Pronouns
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,2,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,0,0,5
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2,0,4


### **<span id="id_secao_prevendo_sobreviventes" style="color:green">SEPARE OS DADOS DE TREINO E TESTE</span>**

Reutilize o método `obter_treino_e_teste()` para dividir o dataframe em treino e teste.

In [18]:
treino, teste = obter_treino_e_teste(obter_dataframe_processado(path=CAMINHO_BASE_TREINAMENTO))

In [19]:
cols_to_predict = ['Pclass', 'Fare', 'SibSp', 'Parch', 'Sex', 'isChild', 'Embarked', 'Age', 'Pronouns']

In [20]:
x_treino = treino[cols_to_predict]
y_treino = treino['Survived']

x_teste = teste[cols_to_predict]
y_teste = teste['Survived']

### **<span id="id_secao_prevendo_sobreviventes" style="color:green">AVALIE DIVERSOS MODELOS EM VALIDAÇÃO CRUZADA</span>**

#### Crie um array de tuplas para armazenar os modelos que serão processados em validação cruzada

In [ ]:
# Modelos sem necessidade de padronização
non_scaled_models = [
    ('GradientBoostingClassifier', GradientBoostingClassifier()),
    ('LGBMClassifier', LGBMClassifier(verbosity=-1)),  
    ('RandomForestClassifier', RandomForestClassifier()),
    ('AdaBoostClassifier', AdaBoostClassifier()),
    ('ExtraTreesClassifier', ExtraTreesClassifier()),
    ('HistGradientBoostingClassifier', HistGradientBoostingClassifier()),
    ('RidgeClassifier', RidgeClassifier()),
    ('DecisionTreeClassifier', DecisionTreeClassifier())
]

In [22]:
# Modelos que precisam de padronização
scaled_models = [
     ('XGBClassifier', XGBClassifier(verbosity=0, objective='binary:hinge')), 
    ('LogisticRegression', LogisticRegression()),
    ('SGDClassifier', SGDClassifier()),
    ('KNeighborsClassifier', KNeighborsClassifier())
]

In [23]:
# Aplicando o Pipeline com StandardScaler apenas aos modelos sensíveis à escala
pipelines = [
    (name, Pipeline([('scaler', StandardScaler()), ('classifier', model)])) 
    for name, model in scaled_models
]

all_models = non_scaled_models + pipelines

##### Razões para Padronizar Dados em Modelos de Machine Learning

1. **Consistência de Escala**
Padronizar dados garante que todas as variáveis estejam na mesma escala, o que é crucial para algoritmos de Machine Learning que dependem da distância entre pontos de dados, como ***regressão linear e k-vizinhos mais próximos***.

2. **Melhoria na Convergência**
Algoritmos de otimização, como gradiente descendente, convergem mais rápido e com mais eficiência quando os dados são padronizados.

3. **Redução de Vieses**
A padronização ajuda a reduzir vieses que podem surgir devido a diferenças de escala, permitindo que o modelo trate todas as variáveis de forma justa.

4. **Melhor Desempenho de Modelos**
Modelos como Support Vector Machines ***(SVM) e Redes Neurais*** beneficiam-se da padronização dos dados, levando a um desempenho superior e resultados mais precisos.

Conclusão
Padronizar dados é um passo fundamental no pré-processamento de dados para Machine Learning, garantindo a consistência e eficácia dos modelos treinados. Certifique-se de incluir esse passo no seu pipeline de dados para obter melhores resultados nos seus projetos de Machine Learning.

---


#### Avalie o desempenho dos modelos

In [24]:
from sklearn.model_selection import KFold

def avaliar_modelos(modelos, X, y, n_splits=10, metric='accuracy'):
    resultados = {}
    
    # Divide a base de dados em 10 folds. 
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    for nome, modelo in modelos:
        scores = []
        
        # Realizar a validação cruzada sem estratificação
        for train_idx, test_idx in kf.split(X):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            
            modelo.fit(X_train, y_train)
            y_pred = modelo.predict(X_test)
            scores.append(accuracy_score(y_test, y_pred))
        
        resultados[nome] = {
            'media': np.mean(scores),
            'desvio_padrao': np.std(scores)
        }
    
    return resultados


##### Descrição do Método `avaliar_modelos`

O método `avaliar_modelos` realiza a validação cruzada em diversos modelos de Machine Learning e retorna a média e o desvio padrão das pontuações de precisão.

### Parâmetros:
- `modelos`: Array de tuplas contendo nomes dos modelos como chaves e instâncias dos modelos como valores.
- `X`: O dataframe contendo as características (features) do dataset.
- `y`: A série ou dataframe contendo os rótulos (labels) do dataset.
- `n_splits` (opcional): Número de folds para a validação cruzada (padrão é 10).
- `metric` (opcional): Métrica de avaliação dos modelos (padrão é 'accuracy').

### Funcionamento:
1. **Divisão da Base de Dados**:
    - Utiliza `KFold` do `sklearn` para dividir a base de dados em `n_splits` folds, com embaralhamento dos dados e uma semente aleatória para reprodutibilidade (random_state).

2. **Iteração sobre os Modelos**:
    - Para cada modelo do array `modelos`, inicializa uma lista `scores` para armazenar as pontuações de cada fold.

3. **Validação Cruzada Sem Estratificação**:
    - Para cada fold, divide os dados de treinamento e teste.
    - Treina o modelo com os dados de treinamento.
    - Prediz os rótulos para os dados de teste.
    - Calcula e armazena a pontuação de precisão para o fold atual.

4. **Cálculo da Média e Desvio Padrão**:
    - Calcula a média e o desvio padrão das pontuações de precisão para cada modelo.

5. **Retorno dos Resultados**:
    - Retorna um dicionário `resultados` contendo a média e o desvio padrão das pontuações para cada modelo.


**Exemplo de Uso**:
```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

modelos = [ 
    ('Random Forest': RandomForestClassifier()),
    ('Logistic Regression': LogisticRegression())
]

resultados = avaliar_modelos(modelos, X, y)
print(resultados)

---

In [ ]:
resultados = avaliar_modelos(all_models, df[cols_to_predict], df.Survived, n_splits=5, metric='accuracy')

for nome, resultado in resultados.items():
    print(f"{nome}: Média = {resultado['media']:.5f}, Desvio Padrão = {resultado['desvio_padrao']:.4f}")

GradientBoostingClassifier: Média = 0.82493, Desvio Padrão = 0.0275
LGBMClassifier: Média = 0.82716, Desvio Padrão = 0.0257
RandomForestClassifier: Média = 0.82715, Desvio Padrão = 0.0149
AdaBoostClassifier: Média = 0.81034, Desvio Padrão = 0.0162
ExtraTreesClassifier: Média = 0.80694, Desvio Padrão = 0.0171
HistGradientBoostingClassifier: Média = 0.83277, Desvio Padrão = 0.0209
RidgeClassifier: Média = 0.81597, Desvio Padrão = 0.0320
DecisionTreeClassifier: Média = 0.77104, Desvio Padrão = 0.0351
XGBClassifier: Média = 0.78898, Desvio Padrão = 0.0324
LogisticRegression: Média = 0.82382, Desvio Padrão = 0.0256
SGDClassifier: Média = 0.79460, Desvio Padrão = 0.0117
KNeighborsClassifier: Média = 0.81144, Desvio Padrão = 0.0230


### **<span style="color:green">COMBINE DIVERSOS MODELOS EM UM ENSAMBLE E FAA PREVISÕES</span>**

In [26]:
# Criando o StackingClassifier
stacking_model = StackingClassifier(
    estimators=all_models,
    final_estimator=LogisticRegression(),  # Meta-aprendiz (pode ser ajustado)
    passthrough=False,  # Se True, inclui as features originais junto com as previsões na meta-aprendizagem
    n_jobs=-1
)

# Exemplo de treinamento
stacking_model.fit(x_treino, y_treino)
y_pred = stacking_model.predict(x_teste)
accuracy_score(y_pred=y_pred, y_true=y_teste)

0.8379888268156425